This code book calls the OpenAI API to classify moral sentiments in posts from the Moral Foundations Reddit Corpus using fine-tuned ChatGPT!

## Load Packages

In [1]:
import openai
import os
import pandas as pd
import numpy as np
from collections import defaultdict
import json

from sklearn.model_selection import train_test_split

import string
import re
remove = string.punctuation
remove = remove.replace("-", "").replace(",", "") # don't remove hyphens
pattern = r"[{}]".format(remove) # create the pattern

import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
def count_tokens(text):
    return len(encoding.encode(text))

import pickle
import time
import logging
from retry import retry
logging.basicConfig()

# Calculate the delay based on your rate limit
rate_limit_per_minute = 3500.0
delay_60 = 60.0 / 60
delay_full = 60.0 / rate_limit_per_minute

## General Parameters

In [82]:
data = "mfrc"
mode = "full"
folder = "../data/preprocessed/"
path = folder + data + "_sample_" + mode + ".csv" # test data
path_train = folder + data + "_train_" + mode + ".csv" # train data

In [92]:
# load training data
df_train_total = pd.read_csv(path_train)

## Functions

In [84]:
# chatGPT parameters
openai.api_key = os.getenv("OPENAI_API_KEY") #add your openai key to the environment
model_engine = "gpt-3.5-turbo-0301"

@retry(delay=5)
def delayed_completion(delay_in_seconds: float = 1, **kwargs):
    """Delay a completion by a specified amount of time."""

    # Sleep for the delay
    time.sleep(delay_in_seconds)

    # Call the Completion API and return the result
    return openai.ChatCompletion.create(**kwargs)

def separate_labels(df, cols):
    def _set_labels(row):
        for label in row["annotations"].split(","):
            if label in cols:
                row[label.strip()] = 1
        return row

    # removing texts with no annotations
    df = df[df.annotations != ''].reset_index(drop=True)
    df = df[~ pd.isna(df.annotations)].reset_index(drop=True)
    for label in cols:
        df[label] = 0
    df = df.apply(_set_labels, axis=1).drop(["annotations"], axis = 1)
    return df

## Load Raw Data and Generate Prompts for Fine-Tuning

In [85]:
def generate_prompts(df):
    def concat_column_names(row):
        response = ', '.join([col for col in df.columns[1:-1] if row[col] == 1])
        if not response:
            response = "non-moral"
        return response
    df['responses'] = df.apply(concat_column_names, axis=1)

    prompts = []
    for (text, response) in zip(df.text, df.responses):
        prompts.append({"messages":[
            {"role": "system", "content": INSTR_TEXT},
            {"role": "user", "content": USER_TEXT + text},
            {"role": "assistant", "content": response}
        ]})

    return prompts

# System role instructions
INSTR_TEXT = "Detect the presence of moral sentiments in a text based on their definitions."
USER_TEXT = "Determine which moral sentiments are expressed in the following text. The text contains "\
"\"care\" if the text is about avoiding emotional and physical damage to another individual, " \
"\"equality\" if the text is about equal treatment and equal outcome for individuals, " \
"\"proportionality\" if the text is about individuals getting rewarded in proportion to their merit or contribution, "\
"\"loyalty\" if the text is about cooperating with ingroups and competing with outgroups, "\
"\"authority\" if the text is about deference toward legitimate authorities and the defense of traditions, "\
"all of which are seen as providing stability and fending off chaos, "\
"\"purity\" if the text is about avoiding bodily and spiritual contamination and degradation, "\
"\"thin morality\" if the text has a moral sentiment but cannot be categorized as either of the above. "\
"Respond only with these words. Respond with all words that apply, comma separated. Here is the text: " 

In [98]:
### Generate prompts
df_train, df_val = train_test_split(df_train_total, random_state=0, test_size=0.2, stratify=df_train_total["non-moral"])
prompts_train = generate_prompts(df_train.copy())
prompts_val = generate_prompts(df_val.copy())

In [100]:
### check prompt
prompts_train[0]

{'messages': [{'role': 'system',
   'content': 'Detect the presence of moral sentiments in a text based on their definitions.'},
  {'role': 'user',
   'content': 'Determine which moral sentiments are expressed in the following text. The text contains "care" if the text is about avoiding emotional and physical damage to another individual, "equality" if the text is about equal treatment and equal outcome for individuals, "proportionality" if the text is about individuals getting rewarded in proportion to their merit or contribution, "loyalty" if the text is about cooperating with ingroups and competing with outgroups, "authority" if the text is about deference toward legitimate authorities and the defense of traditions, all of which are seen as providing stability and fending off chaos, "purity" if the text is about avoiding bodily and spiritual contamination and degradation, "thin morality" if the text has a moral sentiment but cannot be categorized as either of the above. Respond only

In [37]:
# save tuning data:
data_path_train = "../data/moral_fine_tuning_train.jsonl"
with open(data_path_train, "w", encoding='utf-8') as f:
    for prompt in prompts_train:
        json.dump(prompt, f)
        f.write("\n")

data_path_val = "../data/moral_fine_tuning_val.jsonl"
with open(data_path_val, "w", encoding='utf-8') as f:
    for prompt in prompts_val:
        json.dump(prompt, f)
        f.write("\n")

## Data validation and cost estimation

In [12]:
# Load the dataset
data_path = "../data/moral_fine_tuning_train.jsonl"
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 9542
First example:
{'role': 'system', 'content': 'Detect the presence of moral sentiments in a text based on their definitions.'}
{'role': 'user', 'content': 'Determine which moral sentiments are expressed in the following text. The text contains "care" if the text is about avoiding emotional and physical damage to another individual, "equality" if the text is about equal treatment and equal outcome for individuals, "proportionality" if the text is about individuals getting rewarded in proportion to their merit or contribution, "loyalty" if the text is about cooperating with ingroups and competing with outgroups, "authority" if the text is about deference toward legitimate authorities and the defense of traditions, all of which are seen as providing stability and fending off chaos, "purity" if the text is about avoiding bodily and spiritual contamination and degradation, "thin morality" if the text has a moral sentiment but cannot be categorized as either of the above. R

In [13]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [14]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [15]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 221, 538
mean / median: 255.04946552085516, 246.0
p5 / p95: 228.0, 298.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 1, 10
mean / median: 2.6193670090127856, 3.0
p5 / p95: 1.0, 3.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [16]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print(f"By default, you'll be charged ~{round(n_epochs * n_billing_tokens_in_dataset * 0.008/1000, 2)}$")

Dataset has ~2433682 tokens that will be charged for during training
By default, you'll train for 2 epochs on this dataset
By default, you'll be charged for ~4867364 tokens
By default, you'll be charged ~38.94$


## Fine-tune Model

### Upload Training Data

In [44]:
openai.File.create(
  file=open(data_path_train, "rb"),
  purpose='fine-tune'
)

<File file id=file-45wVWmAaT8Ct7tJT110vCFhc at 0x7f196808c770> JSON: {
  "object": "file",
  "id": "file-45wVWmAaT8Ct7tJT110vCFhc",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 12831924,
  "created_at": 1697820790,
  "status": "uploaded",
  "status_details": null
}

In [45]:
openai.File.create(
  file=open(data_path_val, "rb"),
  purpose='fine-tune'
)

<File file id=file-PwQlW6vqTDmsLZjkdrHNGqGe at 0x7f191c4c93a0> JSON: {
  "object": "file",
  "id": "file-PwQlW6vqTDmsLZjkdrHNGqGe",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 3224234,
  "created_at": 1697820847,
  "status": "uploaded",
  "status_details": null
}

### Create Tuning Job

In [53]:
openai.FineTuningJob.create(training_file="file-45wVWmAaT8Ct7tJT110vCFhc", model="gpt-3.5-turbo-0613", suffix="mfrc_tuned", validation_file="file-PwQlW6vqTDmsLZjkdrHNGqGe", hyperparameters={"n_epochs":2})

<FineTuningJob fine_tuning.job id=ftjob-waCyJ9TOZ6ep5OHcrN10fQj4 at 0x7f191e182610> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-waCyJ9TOZ6ep5OHcrN10fQj4",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697821545,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-yVS6iCwS4TFCckmhlIwrbL8m",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-PwQlW6vqTDmsLZjkdrHNGqGe",
  "training_file": "file-45wVWmAaT8Ct7tJT110vCFhc",
  "hyperparameters": {
    "n_epochs": 2
  },
  "trained_tokens": null,
  "error": null
}

### Check up on job (or use online interface)

In [136]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)

# # # Retrieve the state of a fine-tune
# openai.FineTuningJob.retrieve("ftjob-waCyJ9TOZ6ep5OHcrN10fQj4")

<FineTuningJob fine_tuning.job id=ftjob-waCyJ9TOZ6ep5OHcrN10fQj4 at 0x7f1916a2c360> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-waCyJ9TOZ6ep5OHcrN10fQj4",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697821545,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-yVS6iCwS4TFCckmhlIwrbL8m",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-PwQlW6vqTDmsLZjkdrHNGqGe",
  "training_file": "file-45wVWmAaT8Ct7tJT110vCFhc",
  "hyperparameters": {
    "n_epochs": 2
  },
  "trained_tokens": null,
  "error": null
}

## Try fine-tuning API on small subset first (only for test purposes if problems occur)

Use this to test if the fine-tuning service works on a small dataset if necessary (avoids paying for the large training data in case errors occur)

In [113]:
df_train['strata'] = df_train[df_train.columns[1:-1]].apply(lambda row: '_'.join(row.map(str)), axis=1)
df_val['strata'] = df_val[df_val.columns[1:-1]].apply(lambda row: '_'.join(row.map(str)), axis=1)

# Sample from each stratum
samples = []
for stratum, group in df_train.groupby('strata'):
    samples.append(group.sample(frac=0.01, random_state=0))
df_train_mini = pd.concat(samples, axis=0)

samples = []
for stratum, group in df_val.groupby('strata'):
    samples.append(group.sample(frac=0.01, random_state=0))
df_val_mini = pd.concat(samples, axis=0)

# Drop the strata column if you don't need it anymore
df_train_mini = df_train_mini.drop(columns=['strata'])
df_val_mini = df_val_mini.drop(columns=['strata'])

prompts_train_mini = generate_prompts(df_train_mini.copy())
prompts_val_mini = generate_prompts(df_val_mini.copy())

In [115]:
# save mini tuning data:
data_path_train_mini = "../data/moral_fine_tuning_train_mini.jsonl"
with open(data_path_train_mini, "w", encoding='utf-8') as f:
    for prompt in prompts_train_mini:
        json.dump(prompt, f)
        f.write("\n")

data_path_val_mini = "../data/moral_fine_tuning_val_mini.jsonl"
with open(data_path_val_mini, "w", encoding='utf-8') as f:
    for prompt in prompts_val_mini:
        json.dump(prompt, f)
        f.write("\n")

In [117]:
openai.File.create(
  file=open(data_path_train_mini, "rb"),
  purpose='fine-tune'
)

<File file id=file-AeTDS2IoaFK2YBbaesSyHcGE at 0x7f191e1265c0> JSON: {
  "object": "file",
  "id": "file-AeTDS2IoaFK2YBbaesSyHcGE",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 124562,
  "created_at": 1697825882,
  "status": "uploaded",
  "status_details": null
}

In [118]:
openai.File.create(
  file=open(data_path_val_mini, "rb"),
  purpose='fine-tune'
)

<File file id=file-meIj4wsewNZvZtWCCRX0CVtF at 0x7f191e10b650> JSON: {
  "object": "file",
  "id": "file-meIj4wsewNZvZtWCCRX0CVtF",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 28245,
  "created_at": 1697825883,
  "status": "uploaded",
  "status_details": null
}

In [135]:
openai.FineTuningJob.create(training_file="file-AeTDS2IoaFK2YBbaesSyHcGE", model="gpt-3.5-turbo-0613", suffix="mfrc_tuned_mini", validation_file="file-meIj4wsewNZvZtWCCRX0CVtF", hyperparameters={"n_epochs":3})

<FineTuningJob fine_tuning.job id=ftjob-wD3oTJBibcWF1VxmEx9Hzsol at 0x7f191a8fe5c0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-wD3oTJBibcWF1VxmEx9Hzsol",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697826444,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-yVS6iCwS4TFCckmhlIwrbL8m",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-meIj4wsewNZvZtWCCRX0CVtF",
  "training_file": "file-AeTDS2IoaFK2YBbaesSyHcGE",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null,
  "error": null
}

# Run Tuned Model

In [184]:
# load annotation texts
df_test = pd.read_csv(path)
print(df_test.shape)
prompts_test_full = generate_prompts(df_test.copy())
prompts_test = [prompt["messages"][:-1] for prompt in prompts_test_full]
print(len(prompts_test))

(2983, 9)
2983


In [185]:
# Check prompt
prompts_test_full[2]["messages"]

[{'role': 'system',
  'content': 'Detect the presence of moral sentiments in a text based on their definitions.'},
 {'role': 'user',
  'content': 'Determine which moral sentiments are expressed in the following text. The text contains "care" if the text is about avoiding emotional and physical damage to another individual, "equality" if the text is about equal treatment and equal outcome for individuals, "proportionality" if the text is about individuals getting rewarded in proportion to their merit or contribution, "loyalty" if the text is about cooperating with ingroups and competing with outgroups, "authority" if the text is about deference toward legitimate authorities and the defense of traditions, all of which are seen as providing stability and fending off chaos, "purity" if the text is about avoiding bodily and spiritual contamination and degradation, "thin morality" if the text has a moral sentiment but cannot be categorized as either of the above. Respond only with these word

In [139]:
# set tuned model
tuned_model ="ft:gpt-3.5-turbo-0613:personal:mfrc-tuned:8Bpv4Ow4"

## Test Call

In [167]:
APIresponse = delayed_completion(
    delay_in_seconds=delay_full,
    model=tuned_model,
    messages=prompts_test[0],
    temperature=0
    )
response = APIresponse.choices[0].message["content"]
print(response) #works

thin morality


## Run Calls

In [ ]:
responses = []
for i, prompt in enumerate(prompts_test):
    APIresponse = delayed_completion(
        delay_in_seconds=delay_full,
        model=tuned_model,
        messages=prompt,
        temperature=0,
        )
    response = APIresponse.choices[0].message["content"]
    responses.append(response)
    if not i % int(0.1 * len(prompts_test)):
        print(str(int(i/ len(prompts_test)*100)) + "\%")

# clean gpt outputs (for predictions that have imprecise wording, e.g., none for non-moral)
responses_cleaned = [re.sub(pattern, "", x.lower()) if "non" not in x.lower() else "non-moral" for x in responses]

# save as dataframe
new_dic = {}
new_dic["text"] = df_test.text.tolist()
new_dic["annotations_raw"] = responses
df_responses = pd.DataFrame(new_dic)
df_responses.to_csv("../results/predictions/gpt_FT_" + data + "_labels_" + mode + "_raw.csv", index=False)

df_responses_cleaned = df_responses.drop(["annotations_raw"], axis=1).copy()
df_responses_cleaned["annotations"] = responses_cleaned

cols = df.columns[1:].tolist()
df_preds = separate_labels(df_responses_cleaned, cols)
df_preds.to_csv("../results/predictions/gpt_FT_" + data + "_labels_" + mode + ".csv", index=False)